In [6]:
!pip install pytesseract pillow opencv-python
!pip install latexocr


ERROR: Could not find a version that satisfies the requirement latexocr (from versions: none)
ERROR: No matching distribution found for latexocr


In [4]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

from PIL import Image, ImageDraw
import numpy as np

def detect_equation_boxes(image, symbol_threshold=0.4):
    """
    Detect bounding boxes likely to be equations or math expressions.

    Args:
        image: PIL.Image
        symbol_threshold: float, fraction of non-alpha characters to consider a word as math.

    Returns:
        List of bounding boxes: [(x1, y1, x2, y2)]
    """
    d = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
    boxes = []

    for i in range(len(d['text'])):
        word = d['text'][i].strip()
        if not word:
            continue

        non_alpha = sum(not c.isalpha() for c in word)
        frac_non_alpha = non_alpha / len(word)

        if frac_non_alpha >= symbol_threshold:
            x, y, w, h = d['left'][i], d['top'][i], d['width'][i], d['height'][i]
            boxes.append((x, y, x + w, y + h))

    # Optionally: merge nearby boxes
    merged_boxes = merge_boxes(boxes, y_thresh=15)

    return merged_boxes

def merge_boxes(boxes, y_thresh=15):
    """
    Simple merging of overlapping or nearby boxes into bigger regions.
    """
    if not boxes:
        return []

    # Sort by y coordinate
    boxes = sorted(boxes, key=lambda b: (b[1], b[0]))
    merged = []
    current = boxes[0]

    for b in boxes[1:]:
        # If vertical overlap or close
        if b[1] <= current[3] + y_thresh:
            current = (
                min(current[0], b[0]),
                min(current[1], b[1]),
                max(current[2], b[2]),
                max(current[3], b[3]),
            )
        else:
            merged.append(current)
            current = b
    merged.append(current)

    return merged

def draw_boxes(image, boxes, color='red'):
    """
    Draw bounding boxes for visualization.
    """
    draw = ImageDraw.Draw(image)
    for box in boxes:
        draw.rectangle(box, outline=color, width=2)
    return image


In [5]:
from pdf2image import convert_from_path

# Convert PDF page to image
pages = convert_from_path(r'C:\Users\visha\Downloads\Test_math_OCR.pdf', dpi=300)
page_image = pages[0]

# Detect equation boxes
boxes = detect_equation_boxes(page_image)

# Draw and show
output_image = draw_boxes(page_image.copy(), boxes)
output_image.show()
